In [3]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

In [5]:
train_image = pd.read_csv("a/csvTrainImages 13440x1024.csv", header = None)
train_label = pd.read_csv("a/csvTrainLabel 13440x1.csv", header = None)
test_image = pd.read_csv("a/csvTestImages 3360x1024.csv", header = None )
test_label = pd.read_csv("a/csvTestLabel 3360x1.csv", header = None )

In [6]:
# add the lables of the dataset
train_image['label'] = train_label
test_image['label'] = test_label

print(train_image.shape)

(13440, 1025)


In [7]:
class ImageDataset(Dataset):
    
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index, self.data.columns != 'label'].values.astype(np.uint8).reshape(32, 32)
        label = self.data.iloc[index, -1] - 1
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
train_image_dataset = ImageDataset(train_image, transform)

In [9]:
image , label = train_image_dataset.__getitem__(32)

image.size()

torch.Size([1, 32, 32])

In [10]:
loader = DataLoader(train_image_dataset, batch_size= 32, shuffle=True)

In [11]:
class ConvolutionNNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionNNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(1024, 1000)
        self.fc2 = nn.Linear(1000, 28)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
# Train the model
num_epochs = 20
learning_rate = 0.001
total_step = len(loader)
loss_list = []
acc_list = []
model = ConvolutionNNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate);
criterion = nn.CrossEntropyLoss();

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/20], Step [100/420], Loss: 2.2890, Accuracy: 15.62%
Epoch [1/20], Step [200/420], Loss: 1.3825, Accuracy: 56.25%
Epoch [1/20], Step [300/420], Loss: 1.0707, Accuracy: 65.62%
Epoch [1/20], Step [400/420], Loss: 1.1351, Accuracy: 62.50%
Epoch [2/20], Step [100/420], Loss: 0.9157, Accuracy: 75.00%
Epoch [2/20], Step [200/420], Loss: 0.4592, Accuracy: 87.50%
Epoch [2/20], Step [300/420], Loss: 0.4896, Accuracy: 81.25%
Epoch [2/20], Step [400/420], Loss: 0.6778, Accuracy: 71.88%
Epoch [3/20], Step [100/420], Loss: 0.3806, Accuracy: 84.38%
Epoch [3/20], Step [200/420], Loss: 0.1411, Accuracy: 96.88%
Epoch [3/20], Step [300/420], Loss: 0.6904, Accuracy: 78.12%
Epoch [3/20], Step [400/420], Loss: 0.4169, Accuracy: 90.62%
Epoch [4/20], Step [100/420], Loss: 0.3551, Accuracy: 84.38%
Epoch [4/20], Step [200/420], Loss: 0.3652, Accuracy: 87.50%
Epoch [4/20], Step [300/420], Loss: 0.2175, Accuracy: 84.38%
Epoch [4/20], Step [400/420], Loss: 0.2204, Accuracy: 87.50%
Epoch [5/20], Step [100/

In [12]:
test_image_dataset = ImageDataset(test_image, transform)
test_loader = DataLoader(test_image_dataset, batch_size= 1, shuffle=False)
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for (images, labels) in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 3360 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 3360 test images: 94.82142857142857 %
